# Relatório Final — Clusterização de Criptomoedas por Perfil de Risco

O mercado de criptomoedas possui milhares de ativos, com níveis muito distintos de
volatilidade, risco e estabilidade. Para um investidor iniciante, é difícil identificar:

- quais moedas são mais estáveis  
- quais são mais arriscadas  
- quais têm comportamento semelhante  

### **Objetivo do Projeto**
Agrupar criptomoedas por nível de risco utilizando técnicas de **aprendizado não supervisionado**, destacando padrões de comportamento.

Isso ajuda investidores a:

- montar carteiras alinhadas ao seu perfil
- diversificar riscos
- identificar grupos de ativos com comportamento similar

# 1. Metodologia

O projeto segue 5 etapas principais:

---

## **1. Coleta de Dados**
Via API do CoinGecko:
- preços históricos (60 dias) (não considera o "crash" de 10/10)
- volumes
- market cap

---

## **2. Análise Exploratória (EDA)**
Com foco em:
- volatilidade
- correlações
- variações temporais
- identificação de outliers

---

## **3. Feature Engineering**
Para cada moeda, calculamos:

### Métricas financeiras
- **avg_return** → retorno médio
- **volatility** → volatilidade total
- **downside_std** → volatilidade apenas dos retornos negativos
- **abs_return** → retorno absoluto médio (medida forte de instabilidade)

### Métricas fundamentais
- **avg_volume**
- **avg_marketcap**

### Transformações aplicadas
- `log_volume` normalizado (evita dominância do BTC)
- `log_mc` (marketcap log)

### Ajuste importante
O market cap recebeu **peso multiplicativo 5x** na etapa de modelagem, pois é um dos maiores indicadores de estabilidade.

---

## **4. Modelagem**
Modelos avaliados:

- **K-Means (k=5)** → modelo testado e escolhido  
- DBSCAN → teste exploratório

O K-Means teve melhor separação entre grupos.

---

## **5. Visualização**
Foi aplicado PCA (2 componentes) para representar os clusters em 2D.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Carregando os Clusters Finais
Carregamos o arquivo final gerado pela etapa de clusterização para análise e visualização dos resultados.

In [ ]:
df = pd.read_csv("../data/outputs/crypto_clusters.csv", index_col="coin")
df.head()

## Visualização dos Grupos
Visualizamos os clusters no espaço reduzido por PCA, facilitando a interpretação dos grupos formados.

In [ ]:
plt.figure(figsize=(10,6))

sns.scatterplot(
    x=df['pc1'],
    y=df['pc2'],
    hue=df['cluster'],
    palette='Set1',
    s=120
)

for coin, row in df.iterrows():
    plt.text(row['pc1']+0.015, row['pc2']+0.015, coin, fontsize=8)

plt.title("Clusters de Criptomoedas (PCA 2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

## Perfil de Cada Cluster
Calculamos as médias das principais métricas para cada cluster e geramos um heatmap para comparar os perfis dos grupos.

In [ ]:
cluster_profile = df.groupby('cluster').mean()[[
    'avg_return','volatility','downside_std','abs_return',
    'avg_volume','avg_marketcap'
]]

cluster_profile

plt.figure(figsize=(9,6))
sns.heatmap(cluster_profile, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Perfil Médio por Cluster")
plt.show()

## Interpretação dos Resultados
Com base nas métricas analisadas, os clusters podem ser interpretados e utilizados para verificar padrões nas criptomoedas e ajudar a tomar decisões de investimento.


## Conclusões

- O modelo K-Means conseguiu separar criptomoedas em 5 grupos relevantes.
- O PCA mostrou visualmente que os clusters possuem características distintas.
- As métricas de risco foram fundamentais para distinguir comportamento.
- O projeto mostra que é possível aplicar ciência de dados para segmentação
  de ativos financeiros com dados públicos.